In [1]:
import os
import numpy as np
import cv2
import shutil

In [2]:
def mkdir(path):
 
	folder = os.path.exists(path)
 
	if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
		os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径
		print("---  new folder...  ---")
		print("---  OK  ---")
 
	else:
		print("---  There is this folder!  ---")	

## 存储原始数据集

In [3]:
inputDataset = './inputDataset/'
file0 = "./tempDataset1/step0-重新编号后的数据集/"
mkdir(file0) 

i = 0
for root, dirs, files in os.walk(inputDataset, topdown=True):
    for name in files:
        #print((""+os.path.join(root, name)))
        img=cv2.imread((""+os.path.join(root, name)), cv2.IMREAD_GRAYSCALE)
        i = i + 1
        cv2.imwrite(file0 + str(i) + '.bmp', img)

---  There is this folder!  ---


## 随机剪裁

In [4]:
'''
定义裁剪函数，四个参数分别是：
左上角横坐标x0
左上角纵坐标y0
裁剪宽度w
裁剪高度h
'''
crop_image = lambda img, x0, y0, w, h: img[y0:y0+h, x0:x0+w]

In [5]:
'''
随机裁剪
area_ratio为裁剪画面占原画面的比例
hw_vari是扰动占原高宽比的比例范围
'''
def random_crop(img, area_ratio, hw_vari):
    h, w = img.shape[:2]
    hw_delta = np.random.uniform(-hw_vari, hw_vari)
    hw_mult = 1 + hw_delta
    
    # 下标进行裁剪，宽高必须是正整数
    w_crop = int(round(w*np.sqrt(area_ratio*hw_mult)))

    # 裁剪宽度不可超过原图可裁剪宽度
    if w_crop > w:
        w_crop = w

    h_crop = int(round(h*np.sqrt(area_ratio/hw_mult)))
    if h_crop > h:
        h_crop = h

    # 随机生成左上角的位置
    x0 = np.random.randint(0, w-w_crop+1)
    y0 = np.random.randint(0, h-h_crop+1)

    return crop_image(img, x0, y0, w_crop, h_crop)

In [6]:
file1 = "./tempDataset1/step1-图像任意大小剪裁/"
mkdir(file1) 

# test


i = 0
area_ratio = 0.9
hw_vari = 0.1

for root, dirs, files in os.walk(file0, topdown=False):
    files = os.listdir(file0)
    files.sort(key=lambda x:int(x[:-4]))
    for name in files:
        #print((""+os.path.join(root, name)))
        img=cv2.imread((""+os.path.join(root, name)),cv2.IMREAD_GRAYSCALE)
        i = i + 1
        cv2.imwrite(file1+ str(i) + '.bmp',
                    random_crop(img, area_ratio, hw_vari))

---  There is this folder!  ---


## 旋转

In [7]:
file2 = "./tempDataset1/step2-图像任意角度旋转/"
mkdir(file2) 

def rotate_image(img, angle, crop):
    h, w = img.shape[:2]
	
	# 旋转角度的周期是360°
    angle %= 360
	
	# 用OpenCV内置函数计算仿射矩阵
    M_rotate = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
	
	# 得到旋转后的图像
    img_rotated = cv2.warpAffine(img, M_rotate, (w, h))

	# 如果需要裁剪去除黑边
    if crop:
        angle_crop = angle % 180             	    # 对于裁剪角度的等效周期是180°
        if angle_crop > 90:                        	# 并且关于90°对称
            angle_crop = 180 - angle_crop
		
        theta = angle_crop * np.pi / 180.0    		# 转化角度为弧度
        hw_ratio = float(h) / float(w)    		    # 计算高宽比
		

        tan_theta = np.tan(theta)                   # 计算裁剪边长系数的分子项
        numerator = np.cos(theta) + np.sin(theta) * tan_theta
		

        r = hw_ratio if h > w else 1 / hw_ratio		# 计算分母项中和宽高比相关的项
        denominator = r * tan_theta + 1		 		# 计算分母项

        crop_mult = numerator / denominator			# 计算最终的边长系数
        w_crop = int(round(crop_mult*w))			# 得到裁剪区域
        h_crop = int(round(crop_mult*h))
        x0 = int((w-w_crop)/2)
        y0 = int((h-h_crop)/2)
        img_rotated = crop_image(img_rotated, x0, y0, w_crop, h_crop)
    return img_rotated

---  There is this folder!  ---


In [8]:
# test
i = 0
for root, dirs, files in os.walk(file1, topdown=False):
    files = os.listdir(file1)
    files.sort(key=lambda x:int(x[:-4]))
    for name in files:
        img=cv2.imread((""+os.path.join(root, name)), cv2.IMREAD_GRAYSCALE)
        angle = 180
        crop = 0.1
        rotatedImg = rotate_image(img, angle, crop)
        i = i + 1
        cv2.imwrite(file2 + str(i) + '.bmp',rotatedImg)

## 伽马变换

In [9]:
'''
定义gamma变换函数：
gamma就是Gamma
'''
def gamma_transform(img, gamma):
    gamma_table = [np.power(x / 255.0, gamma) * 255.0 for x in range(256)]
    gamma_table = np.round(np.array(gamma_table)).astype(np.uint8)
    return cv2.LUT(img, gamma_table)

'''
随机gamma变换
gamma_vari是Gamma变化的范围[1/gamma_vari, gamma_vari)
'''
def random_gamma_transform(img, gamma_vari):
    log_gamma_vari = np.log(gamma_vari)
    alpha = np.random.uniform(-log_gamma_vari, log_gamma_vari)
    gamma = np.exp(alpha)
    return gamma_transform(img, gamma)

In [10]:
file3 = "./tempDataset1/step3-图像伽马变换/"
mkdir(file3) 

---  There is this folder!  ---


In [11]:
# test
img = cv2.imread(inputDataset)

# test
i = 0
for root, dirs, files in os.walk(file2, topdown=False):
    files = os.listdir(file2)
    files.sort(key=lambda x:int(x[:-4]))
    for name in files:
        img=cv2.imread((""+os.path.join(root, name)),cv2.IMREAD_GRAYSCALE)
        gamma_vari = 0.8
        gammaedImg = random_gamma_transform(img, gamma_vari)
        i = i + 1
        cv2.imwrite(file3 + str(i) + '.bmp',gammaedImg)
    for name in dirs:
        print(os.path.join(root, name))

## 数据集合

In [12]:
file4 = "./lableDataset/"
mkdir(file4) 

# test
i = 0
for root, dirs, files in os.walk("./tempDataset1/", topdown=False):
    for name in files:
        img=cv2.imread((""+os.path.join(root, name)),cv2.IMREAD_GRAYSCALE)
        dst=cv2.resize(img,(640,480)) #改变图片尺寸
        i = i + 1
        cv2.imwrite(file4 + str(i) + '.bmp',dst)
    for name in dirs:
        print(os.path.join(root, name))

---  There is this folder!  ---
./tempDataset1/step0-重新编号后的数据集
./tempDataset1/step1-图像任意大小剪裁
./tempDataset1/step3-图像伽马变换
./tempDataset1/step2-图像任意角度旋转


## 图像模糊

In [13]:
file5 = "./tempDataset2-高斯模糊/"
mkdir(file5) 

# test
i = 0
for root, dirs, files in os.walk(file4, topdown=False):
    files = os.listdir(file4)
    files.sort(key=lambda x:int(x[:-4]))
    for name in files:
        img=cv2.imread((""+os.path.join(root, name)),cv2.IMREAD_GRAYSCALE)
        blurImg = cv2.GaussianBlur(img,(3,3),1)
        i = i + 1
        cv2.imwrite(file5 + str(i) + '.bmp',blurImg)
    for name in dirs:
        print(os.path.join(root, name))

---  There is this folder!  ---


## 图像resize得到输出

In [14]:
file6 = "./trainDataset/"
mkdir(file6) 

i = 0
for root, dirs, files in os.walk(file5, topdown=False):   
    files = os.listdir(file5)
    files.sort(key=lambda x:int(x[:-4]))
    for name in files:
        img=cv2.imread((""+os.path.join(root, name)),cv2.IMREAD_GRAYSCALE)
        dst=cv2.resize(img,(640,480)) #改变图片尺寸
        i = i + 1
        cv2.imwrite(file6+ str(i) + '.bmp',dst)

---  There is this folder!  ---
